# Dataset for the Arithmetic Equation Correction (AEC) Task

In [7]:
# dependency
# public
import os
import numpy as np
from tqdm import tqdm
# private
from utils import save_txt

In [11]:
# class for data generation of the Arithmetic Equation Correction (AEC) problem 
class ArithmeticEquationCorrection(): 
    """docstring for ArithmeticEquationCorrection"""
    def __init__(self, operators, num_size):
        super().__init__()
        self.operators = operators
        self.pos_digits = np.arange(2, num_size+2).tolist()
        self.neg_digits = np.arange(-num_size, -1).tolist()
        self.digits = self.pos_digits + self.neg_digits
    
    def gen_base_dict(self):
        return {str(i):[] for i in self.pos_digits}
    
    def expand_base_dict(self):
        for a in self.digits:
            for o in self.operators:
                for b in self.pos_digits:
                    try:
                        e = [str(a), o, str(b)]
                        v = str(eval(''.join(e)))
                        e[0] = e[0].replace('-', '- ')
                        e = ' '.join(list(e))
                        if v in self.base_dict:
                            self.base_dict[v].append('( {} )'.format(e))
                    except:
                        pass
    
    def gen_operation(self, seq_len):
        if seq_len == 1:
            a = np.random.choice(self.digits)
            return [str(a)]
        else:
            left_side  = self.gen_operation(seq_len-1)
            o = np.random.choice(self.operators)
            b = np.random.choice(self.pos_digits)
            return left_side + [o, str(b)]
    
    def gen_operation_list(self, seq_len, data_size):
        # to control the data size
        operations_pool = set()
        for i in tqdm(range(data_size)):
            while True: 
                # to avoid duplicates
                operation = self.gen_operation(seq_len) 
                if ''.join(operation) in operations_pool: 
                    continue
                else:
                    operations_pool.add(''.join(operation)) 
                # to avoid zero division error
                try: 
                    # flost to int to string
                    value = eval(''.join(operation))
                    if value % 1 != 0.: 
                        continue
                    else:
                        value = str(int(value))
                        # to keep vocab size
                        if value in self.value_dict: 
                            self.value_dict[value].append(operation)
                            break
                except: 
                    pass
    
    def gen_equation_list(self):
        ys = []
        for v in self.value_dict:
            for y in self.value_dict[v]:
                y = y[0].replace('-', '- ').split() + y[1:]
                y += ["=="] + [v]
                ys.append(' '.join(y))
        return ys
    
    def replace_numbers(self, ys):
        xs = []
        for y in ys:
            y = y.split()
            num_idx = [i for i, token in enumerate(y) if token.isdigit()]
            num_to_replace = np.random.choice(range(len(num_idx)+1))
            idx_to_replace = np.random.choice(num_idx, num_to_replace, False)
            for i in idx_to_replace:
                y[i] = np.random.choice(self.base_dict[y[i]])
            xs.append(' '.join(y))
        return xs
                
    def generate(self, seq_len, data_size):
        # input sequences, output sequences
        xs, ys = [], []
        self.base_dict = self.gen_base_dict()
        self.value_dict = self.gen_base_dict()
        self.expand_base_dict()
        self.gen_operation_list(
            seq_len=seq_len, 
            data_size=data_size)
        ys = self.gen_equation_list()
        xs = self.replace_numbers(ys)
        
        return xs, ys

In [16]:
# data parameters 
seq_len = 5
num_size = 100
data_size = 100
operators = ['+', '-', '*', '/']

In [17]:
aes = ArithmeticEquationCorrection(operators, num_size)
xs, ys = aes.generate(seq_len-1, data_size)

100%|██████████| 100/100 [00:00<00:00, 473.23it/s]


In [18]:
len(xs)

100

In [19]:
xs[:15]

['- 2 * 56 + 63 + 52 == 3',
 '- ( 9 + 30 ) - ( - 62 + 65 ) + ( 90 - 20 ) - 24 == 4',
 '25 - 28 + ( 70 - 46 ) - ( - 15 + 31 ) == 5',
 '- ( 46 - 23 ) - ( 7 + 80 ) + ( 14 + 55 ) + ( 89 - 41 ) == ( - 78 + 85 )',
 '- 34 * 2 + 36 + 39 == ( - 28 + 35 )',
 '- ( 69 - 54 ) + ( - 17 + 87 ) - ( - 43 + 97 ) + ( 31 - 23 ) == 9',
 '- 16 + 34 - ( 94 - 18 ) + ( - 26 + 93 ) == ( - 46 + 55 )',
 '- 66 / ( 43 - 32 ) + ( 76 - 27 ) - 33 == 10',
 '- 15 + 31 - 27 + ( - 76 + 97 ) == 10',
 '- 43 - 34 - ( - 35 + 45 ) + ( 45 + 53 ) == ( - 44 + 55 )',
 '35 - 22 + 68 - 69 == 12',
 '- ( - 44 + 54 ) * ( - 13 + 27 ) + ( 3 * 22 ) + ( 74 + 13 ) == ( - 19 + 32 )',
 '- 12 - 10 - 60 + 95 == 13',
 '( 70 - 32 ) - ( - 22 + 56 ) + ( 69 - 10 ) - 47 == ( 76 - 60 )',
 '( 36 - 22 ) + ( - 4 + 10 ) - ( - 55 + 84 ) + 26 == ( - 15 + 32 )']

In [21]:
len(ys)

200

In [22]:
ys[:15]

['6 - 9 + 5 == 2',
 '- 2 - 2 + 6 == 2',
 '- 6 + 11 - 3 == 2',
 '- 2 - 4 + 8 == 2',
 '4 / 10 * 5 == 2',
 '11 - 7 - 2 == 2',
 '- 3 + 10 / 2 == 2',
 '9 - 3 - 4 == 2',
 '9 + 4 - 11 == 2',
 '5 / 5 * 2 == 2',
 '10 - 10 + 2 == 2',
 '8 / 2 / 2 == 2',
 '- 8 / 8 + 3 == 2',
 '11 * 2 / 11 == 2',
 '6 - 2 - 2 == 2']

In [34]:
# train val test split
dataset = np.array([(x, y) for x, y in zip(xs, ys)])
data_size = dataset.shape[0]
indices = np.random.permutation(data_size)
train_size = int(0.7*data_size)
val_size = int(0.15*data_size)
test_size = data_size - train_size - val_size
train_idxes = indices[:train_size]
val_idxes = indices[train_size: train_size+val_size]
test_idxes = indices[train_size+val_size:]
trainset = dataset[train_idxes]
valset = dataset[val_idxes]
testset = dataset[test_idxes]
print('train size', train_size, trainset.shape)
print('val size', val_size, valset.shape)
print('test size', test_size, testset.shape)

train size 7000 (7000, 2)
val size 1500 (1500, 2)
test size 1500 (1500, 2)


In [40]:
# to save dataset
outdir = 'aes/'
outdir = os.path.join(
    outdir, 
    'num_size_{}'.format(num_size), 
    'seq_len_{}'.format(seq_len+1), 
    'data_size_{}'.format(data_size))
if not os.path.exists(outdir): 
    os.makedirs(outdir)
outdir

'aes/num_size_10/seq_len_5/data_size_10000'

In [41]:
save_txt(os.path.join(outdir, 'train_x.txt'), trainset[:, 0])
save_txt(os.path.join(outdir, 'train_y.txt'), trainset[:, 1])
save_txt(os.path.join(outdir, 'val_x.txt'), valset[:, 0])
save_txt(os.path.join(outdir, 'val_y.txt'), valset[:, 1])
save_txt(os.path.join(outdir, 'test_x.txt'), testset[:, 0])
save_txt(os.path.join(outdir, 'test_y.txt'), testset[:, 1])